<a href="https://colab.research.google.com/github/202515026woorim/-/blob/main/04_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#필수 라이브러리 설치
!apt-get install -y libgl1-mesa-glx
!pip install -U yt-dlp ultralytics opencv-python
import yt_dlp
import cv2
import time
import os
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

# YouTube 영상 다운로드
def download_youtube_video(url, output_path="traffic.mp4"):
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best',
        'outtmpl': output_path,
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        print(f"✅ YouTube 영상 다운로드 완료: {output_path}")
    except Exception as e:
        print(f"❌ YouTube 다운로드 오류: {e}")

# 영상 URL 설정 및 다운로드
video_url = 'https://www.youtube.com/shorts/NJ4kfYqbvEQ'
download_youtube_video(video_url, "/content/traffic.mp4")

# YOLOv11x 모델 로드 (사전에 업로드 필요)
model = YOLO('yolo11x.pt')

# 영상 열기
cap = cv2.VideoCapture("/content/traffic.mp4")

if not cap.isOpened():
    print("❌ 비디오 파일을 열 수 없습니다. 다운로드가 실패했을 가능성이 있습니다.")
else:
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps
    print(f"🎥 영상 정보: {duration:.2f}초, {fps:.2f} FPS, 총 {total_frames} 프레임")

    out = cv2.VideoWriter('/content/output.mp4', cv2.VideoWriter_fourcc(*'MP4V'), int(fps), (frame_width, frame_height))
    if not out.isOpened():
        print("❌ VideoWriter가 정상적으로 열리지 않았습니다. 코덱을 확인하세요.")
    else:
        print("✅ VideoWriter 초기화 성공")

    car_ids = set()
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model.track(frame, persist=True, tracker="bytetrack.yaml", classes=[2, 3, 5, 7], conf=0.5)

        if results and results[0].boxes.id is not None:
            for box_id in results[0].boxes.id.cpu().numpy().astype(int):
                car_ids.add(box_id)

        annotated_frame = results[0].plot() if results else frame
        out.write(annotated_frame)

    cap.release()
    out.release()

    print(f"🔹 전체 영상에서 통과한 차량 수: {len(car_ids)}대")
    print("\n📌 결과 영상 확인 방법:")
    print("1. 좌측 폴더 아이콘 📁 클릭")
    print("2. `/content/output.mp4` 파일 우클릭 → [다운로드] 또는 [미리보기]")

    output_path = "/content/output.mp4"
    if os.path.exists(output_path):
        print("✅ output.mp4 파일이 정상적으로 저장되었습니다.")
        from google.colab import files
        files.download(output_path)
    else:
        print("❌ output.mp4 파일이 생성되지 않았습니다. OpenCV 설정을 확인하세요.")

# 2. 영상 다운로드
!yt-dlp -f bestvideo+bestaudio --merge-output-format mp4 https://youtube.com/shorts/70KzqLyV83c?si=F5aMu2wJ1Q8PaC-Y -o "video.mp4"

# 3. YOLOv11 모델 (사실은 YOLOv8n, 차후에 교체 가능)
from ultralytics import YOLO
import cv2
import numpy as np

# 💡 나중에 YOLOv11이 실제로 나오면 여기 모델만 교체하면 됩니다
model = YOLO('yolov8n.pt')  # 'yolov11.pt' 자리로 대체 가능

# 4. 영상 열기
cap = cv2.VideoCapture('video.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# 5. 결과 저장 준비
out = cv2.VideoWriter('yolov11_output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# 프레임 분석 설정
skip_frame = 3
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_count % skip_frame == 0:
        # 차량 클래스만 분석
        results = model(frame, classes=[2, 3, 5, 7])
        annotated = results[0].plot()
        if annotated is None or not isinstance(annotated, np.ndarray):
            annotated = frame
    else:
        annotated = frame

    out.write(annotated)
    frame_count += 1

cap.release()
out.release()

print("✅ YOLOv11 스타일 분석 완료! 'yolov11_output.mp4'로 저장됨")
from IPython.display import Video
Video('yolov11_output.mp4', embed=True)

if os.path.exists("yolov11_output.mp4"):
    print("✅ 영상이 성공적으로 저장되었습니다.")
    print("파일 크기:", os.path.getsize("yolov11_output.mp4"), "bytes")
else:
    print("❌ 영상 파일이 없습니다.")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgl1-mesa-glx is already the newest version (23.0.4-0ubuntu1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
[youtube] Extracting URL: https://www.youtube.com/shorts/NJ4kfYqbvEQ
[youtube] NJ4kfYqbvEQ: Downloading webpage
[youtube] NJ4kfYqbvEQ: Downloading tv client config
[youtube] NJ4kfYqbvEQ: Downloading player 73381ccc-main
[youtube] NJ4kfYqbvEQ: Downloading tv player API JSON
[youtube] NJ4kfYqbvEQ: Downloading ios player API JSON
[youtube] NJ4kfYqbvEQ: Downloading m3u8 information
[info] NJ4kfYqbvEQ: Downloading 1 format(s): 614+140
[download] /content/traffic.mp4 has already been downloaded
✅ YouTube 영상 다운로드 완료: /content/traffic.mp4
🎥 영상 정보: 6.38초, 23.97 FPS, 총 153 프레임
✅ VideoWriter 초기화 성공

0: 640x384 5 cars, 1 truck, 3685.9ms
Speed: 4.7ms preprocess, 3685.9ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 5 cars, 1 truck, 2425.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[youtube] Extracting URL: https://youtube.com/shorts/70KzqLyV83c?si=F5aMu2wJ1Q8PaC-Y
[youtube] 70KzqLyV83c: Downloading webpage
[youtube] 70KzqLyV83c: Downloading tv client config
[youtube] 70KzqLyV83c: Downloading player 73381ccc-main
[youtube] 70KzqLyV83c: Downloading tv player API JSON
[youtube] 70KzqLyV83c: Downloading ios player API JSON
[youtube] 70KzqLyV83c: Downloading m3u8 information
[info] 70KzqLyV83c: Downloading 1 format(s): 399+251
[download] video.mp4 has already been downloaded

0: 640x384 10 cars, 1 truck, 155.3ms
Speed: 5.0ms preprocess, 155.3ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 10 cars, 1 truck, 147.4ms
Speed: 4.3ms preprocess, 147.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 7 cars, 151.1ms
Speed: 4.2ms preprocess, 151.1ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 7 cars, 196.7ms
Speed: 4.4ms preprocess, 196.7ms inference, 2.2ms postprocess per image at 